In [1]:
import numpy as np
import argparse
import cv2
import string
from keras.models import Sequential, load_model
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
from nltk.tag import pos_tag
import json
from fuzzywuzzy import fuzz

c:\program files\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
c:\program files\python36\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
KERAS_Alphabet_model_File_Path = r'./Keras_Alphabet_Model.h5'
Alphabet_Mapping_List = list(string.ascii_uppercase)
kernel = np.ones((3,3),np.uint8)

global X, Y, W, H, temp_Index, Predicted_Text, words_To_Hide, list_Character_Positions, list_Words_To_Hide, word_Matching_Thresh_Value


X=Y=W=H=0
temp_Index = 0
Predicted_Text = ''
words_To_Hide = ''
list_Character_Positions = []
list_Words_To_Hide = []
word_Matching_Thresh_Value = 80


In [3]:
try:
    KERAS_Alphabet_Model = load_model(KERAS_Alphabet_model_File_Path)
except:
    KERAS_Model = load_model(KERAS_model_File_Path)

In [4]:
def Get_Processed_Image(input_Image):
    #Gray Image
    input_Image = cv2.cvtColor(input_Image, cv2.COLOR_BGR2GRAY)

    #Blurring
    input_Image = cv2.GaussianBlur(input_Image,(5,5),0)
    
    #Otsu_Threshold_GaussianBlur_Image
    input_Image = cv2.threshold(input_Image,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    
    #Take Negative
    input_Image = cv2.bitwise_not(input_Image)

    #Closing To Remove Noise
    input_Image = cv2.morphologyEx(input_Image, cv2.MORPH_CLOSE, kernel)

    return input_Image



## Dilation for thickening image
def Get_Dilated_Image(input_Image, number):
    kernel = np.ones((5,number),np.uint8)
    
    return (cv2.dilate(input_Image, kernel, iterations=3))

        
        

def Get_Countours(input_Image):
    return  (cv2.findContours(input_Image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[1])

    return contours


def Sort_Countours(Contours, method="left-to-right"):
    # initialize the reverse flag and sort index
    reverse = False
    i = 0

    # handle if we need to sort in reverse
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True

    # handle if we are sorting against the y-coordinate rather than
    # the x-coordinate of the bounding box
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1

    # construct the list of bounding boxes and sort them from top to
    # bottom
    boundingBoxes = [cv2.boundingRect(c) for c in Contours]
    (Contours, boundingBoxes) = zip(*sorted(zip(Contours, boundingBoxes),
                                        key=lambda b:b[1][i], reverse=reverse))

    # return the list of sorted contours and bounding boxes
    return (Contours)

def getNewResizedImage(input_Image, Image_Size):
    height,width = input_Image.shape
    #print (height, width)

    if width > height:
        aspect_Ratio = (float)(width/height)
        width = 20
        height = round(width/aspect_Ratio)
    else:
        aspect_Ratio = (float)(height/width)
        height = 20
        width = round(height/aspect_Ratio)
        
    input_Image = cv2.resize(input_Image, (width,height), interpolation = cv2.INTER_AREA )
    
    height,width = input_Image.shape
    
    number_Of_Column_To_Add = 28-width
    temp_Column = np.zeros( (height , int(number_Of_Column_To_Add/2)), dtype = np.uint8)
    input_Image = np.append(temp_Column, input_Image, axis=1)
    input_Image = np.append(input_Image, temp_Column, axis=1)


    height,width = input_Image.shape

    number_Of_Row_To_Add = 28-height
    temp_Row= np.zeros( (int(number_Of_Row_To_Add/2) , width ), dtype = np.uint8)
    input_Image = np.concatenate((temp_Row,input_Image))
    input_Image = np.concatenate((input_Image,temp_Row))

    return cv2.resize(input_Image, (Image_Size,Image_Size), interpolation = cv2.INTER_AREA )


def Get_Average_Space_From_Image_Line(contours):
    total_space = 0
    number_Of_Character = 0
    last_Point = -1
    avg_Space = 0
    for cnt in contours:
        x,y,w,h = cv2.boundingRect(cnt)

        if w > 30 and h > 30 and x > 0 and y > 0:
            if last_Point == -1:
                last_Point = 0
            else:
                total_space = total_space + ( x - last_Point )
            last_Point = x + w

            number_Of_Character = number_Of_Character + 1

    avg_Space = int (total_space/number_Of_Character)

    total_space = 0
    last_Point = -1
    number_Of_Character = 0

##    for cnt in contours:
##        x,y,w,h = cv2.boundingRect(cnt)
##
##        if w > 30 and h > 30 and x > 0 and y > 0:
##            if last_Point == -1:
##                last_Point = 0
##            else:
##                space = ( (x + w) - last_Point )
##                if space < avg_Space:
##                    total_space = total_space + space
##                    number_Of_Character = number_Of_Character + 1
##            last_Point = x + w
##            
##    avg_Space = int (total_space/number_Of_Character)

    return avg_Space



def Get_Text_From_Image(Image, contours):
    global count,X,Y,W,H, list_Character_Positions
    KERAS_Alphabet_Prediction = ''

    total_space = 0
    number_Of_Character = 0
    last_Point = -1

    #avg_Space = Get_Average_Space_From_Image_Line(contours)
    last_Point = -1
    total_space = 0
    #print (avg_Space)
    
    Word_Dilated_Image = Get_Dilated_Image(Image,20)
    Word_Contours = Get_Countours(Word_Dilated_Image)
    Word_Contours = Sort_Countours(Word_Contours,"left-to-right")
    last_Word_Contour_Index = 0
    Word_X, Word_Y, Word_W, Word_H = cv2.boundingRect(Word_Contours[last_Word_Contour_Index])
    last_Word_Contour_Max_X_Range = Word_X + Word_W
    # i.e X + W
            

    for cnt in contours:
        x,y,w,h = cv2.boundingRect(cnt)
        give_Space = False
        #print (x,y,w,h)

        # Reject if Contour is not of desired size (too small)
        if w > 30 and h > 30 and x > 0 and y > 0:

#             ## Spacing based on average space between character
#             if last_Point == -1:
#                 last_Point = 0
#             else:
#                 total_space = ( x - last_Point )

#             last_Point = x + w

#             if total_space > (avg_Space*1.3):
#                 give_Space = True
            
            ## Spacing based on word formation
            if x > last_Word_Contour_Max_X_Range:
                give_Space = True
                last_Word_Contour_Index = last_Word_Contour_Index + 1
                Word_X, Word_Y, Word_W, Word_H = cv2.boundingRect(Word_Contours[last_Word_Contour_Index])
                last_Word_Contour_Max_X_Range = Word_X + Word_W
                # i.e X + W
                
            if give_Space == True:
                KERAS_Alphabet_Prediction = KERAS_Alphabet_Prediction + " "
                list_Character_Positions.append((-1,-1,-1,-1," "))
                

            resize_image = getNewResizedImage(Image[y:y+h, x:x+w] , 28)
            resize_image = resize_image.flatten()

            temp_Index = int(KERAS_Alphabet_Model.predict_classes(resize_image.reshape(1,784)/255.0)[0])
            alphabet_probability = (KERAS_Alphabet_Model.predict_proba(resize_image.reshape(1,784)/255.0))
            sort_alphabet_probability = -np.sort(-alphabet_probability)

            KERAS_Alphabet_Prediction = KERAS_Alphabet_Prediction + Alphabet_Mapping_List[int(temp_Index)]

            list_Character_Positions.append((x+X,y+Y,w,h,str(Alphabet_Mapping_List[int(temp_Index)])))

##            if sort_alphabet_probability[0,0] > 0.99:
##                KERAS_Alphabet_Prediction = KERAS_Alphabet_Prediction + Alphabet_Mapping_List[int(temp_Index)]
##            else:
##                alternate_Probable_Alphabet = Alphabet_Mapping_List[int(np.where(alphabet_probability == sort_alphabet_probability[0,1])[1][0])]
##                KERAS_Alphabet_Prediction = KERAS_Alphabet_Prediction + "[" + Alphabet_Mapping_List[int(temp_Index)] + "," + alternate_Probable_Alphabet + "]"

    list_Character_Positions.append((-1,-1,-1,-1," "))

    return KERAS_Alphabet_Prediction

In [5]:
def Get_Similar_List_Of_Words_To_Hide(list_Words_To_Hide, Predicted_Text):
    list_Similar_Words = []
    list_Predicted_Text_Words = Predicted_Text.split(" ")
    global word_Matching_Thresh_Value
    
    for word_To_Hide in list_Words_To_Hide:
        for predicted_Word in list_Predicted_Text_Words:
            word_Probability = fuzz.ratio(word_To_Hide,predicted_Word)
            if word_Probability > word_Matching_Thresh_Value:
                list_Similar_Words.append(predicted_Word)
                
    return list_Similar_Words

In [6]:
def Get_List_Dimension_Of_Words_Hide(Predicted_Text, list_Character_Positions,  list_Words_To_Hide):
    list_Dimension_Of_Words_Hide = []
    
    for word in list_Words_To_Hide:
        count_Of_Word = Predicted_Text.count(word)
        word_index = -1
        char_index = 0
        
        for index in range(0,count_Of_Word):
            word_index = Predicted_Text.find(word, word_index + 1)
            Min_X = Min_Y = 1000000
            Max_X = Max_Y = 0
            
            for i in range (word_index, word_index + len(word)):
                char_Pos = list_Character_Positions[i]
                x,y,w,h = char_Pos[0],char_Pos[1],char_Pos[2],char_Pos[3]
                
                if Min_X > x and x > -1:
                    Min_X = x
                if Min_Y > y and y > -1:
                    Min_Y = y
                if Max_X < (x + w):
                    Max_X = (x + w)
                if Max_Y < (y + h):
                    Max_Y = (y + h)
                    
                char_index = char_index + 1
            
            list_Dimension_Of_Words_Hide.append((Min_X, Min_Y, Max_X, Max_Y, word))
                
    return list_Dimension_Of_Words_Hide
    
    

In [9]:
def Get_Character_Position_From_Image(image_File_Path, Words_To_Hide = ""):
    if not (os.path.isfile(image_File_Path)):
        return "Invalid image file path"
    
    image = cv2.imread(image_File_Path)
    image = Get_Processed_Image(image)
    
    image_With_Lines = Get_Dilated_Image(image, 60)
    
    contours = Get_Countours(image_With_Lines)
    contours = Sort_Countours(contours, "top-to-bottom")
    
    global X, Y, W, H, temp_Index, Predicted_Text, words_To_Hide, list_Character_Positions, list_Words_To_Hide, word_Matching_Thresh_Value
    
    Predicted_Text = ''
    temp_Index = ''
    X=Y=W=H=0
    list_Character_Positions = []
    list_Words_To_Hide = []
    word_Matching_Thresh_Value = 80
    words_To_Hide = Words_To_Hide
    
    for line_Area in contours:
        x,y,w,h = cv2.boundingRect(line_Area)
        X,Y,W,H = x,y,w,h
        line_Image = image[y:y+h, x:x+w]
        
        line_Contours = Get_Countours(line_Image)
        line_Contours = Sort_Countours(line_Contours,"left-to-right")
        
        Predicted_Text = Predicted_Text + " " + (Get_Text_From_Image(line_Image, line_Contours))
    
    Predicted_Text = Predicted_Text.strip(" ")
    
    json_list_Character_Positions = json.dumps(list_Character_Positions)
    
    #print (json_list_Character_Positions)
    
    if not words_To_Hide == "":
        while "  " in words_To_Hide:
            words_To_Hide =  words_To_Hide.replace("  "," ")
        if words_To_Hide == " ":
            return json_list_Character_Positions
        
        
        list_Words_To_Hide = words_To_Hide.upper().strip().split(" ")
        list_Similar_Words = Get_Similar_List_Of_Words_To_Hide(list_Words_To_Hide, Predicted_Text)
        
        for similar_Word in list_Similar_Words:
            if not similar_Word in list_Words_To_Hide:
                list_Words_To_Hide.append(similar_Word)
        
        
        list_Dimension_Of_Words_Hide = Get_List_Dimension_Of_Words_Hide(Predicted_Text, list_Character_Positions,  list_Words_To_Hide)
        
        json_list_Dimension_Of_Words_Hide = json.dumps(list_Dimension_Of_Words_Hide)
        
        return json_list_Character_Positions, json_list_Dimension_Of_Words_Hide
    
    else:
        return json_list_Character_Positions

In [10]:
words_To_Hide = "  Hemant  MASTEK  Project    Shelar    Working  kiosk"
a, b = (Get_Character_Position_From_Image (r'C:\Users\sachin13390\Desktop\Self-Declaration.jpg',words_To_Hide))

print (b)

[[1245, 171, 1782, 251, "HEMANT"], [1865, 379, 2383, 458, "MASTEK"], [827, 376, 1474, 457, "WORKING"], [1352, 797, 1947, 888, "WORKING"], [438, 1295, 963, 1395, "PROIECT"], [1890, 181, 2414, 265, "SMELAR"]]
